In [1]:
import rds_config
from sqlalchemy import create_engine

#rds settings
db_host  = rds_config.db_host
db_username = rds_config.db_username
db_password = rds_config.db_password
db_name = rds_config.db_name

#한글 깨짐 현상 해결
#http://yongho1037.tistory.com/570
SQLALCHEMY_DATABASE_URI = 'mysql+pymysql://' + db_username + ':' + db_password + '@' + db_host + '/' + db_name + '?charset=utf8'

engine = create_engine(SQLALCHEMY_DATABASE_URI)
connection = engine.connect()

In [2]:
import pandas as pd
sql = "SELECT date, substring(date, 6,2) as month, media, title, org_count, naver_count, org_count+naver_count as total_count, section FROM newscrawl WHERE org_count>0 or naver_count>0 "
df = pd.read_sql(sql, connection)
#pd.read_csv("dcdata10.csv")

In [3]:
len(df)

449603

In [4]:
df.tail(10)

,date,month,media,title,org_count,naver_count,total_count,section
449593,2016-12-14,12,jtbc,"박 대통령 '공짜 진료' 논란…특검, 뇌물죄 적용 고심",2,12,14,사회
449594,2016-12-14,12,jtbc,"[오늘] 12월 14일…'최순실 육성' 공개, 분명한 메시지",4,34,38,정치
449595,2016-12-14,12,jtbc,[앵커브리핑] 눈먼 자들의 국가…감출 수 없는 진실들,6902,53,6955,정치
449596,2016-12-14,12,jtbc,다시 한 번 또 봐도…'무능했던' 청와대의 7시간 대응,68,23,91,정치
449597,2016-12-14,12,jtbc,"최순실 ""조작으로 몰아라""…태블릿PC 소유 자백한 셈",166,30,196,사회
449598,2016-12-14,12,jtbc,[밀착카메라] 청문회 준비 어떻게…국조 위원들 24시,14,37,51,정치
449599,2016-12-14,12,jtbc,다급했던 최순실 통화…JTBC 보도 사흘 뒤 '은폐 지시',9175,50,9225,사회
449600,2016-12-14,12,jtbc,"[비하인드 뉴스] 대통령 미용사, 버킹엄궁도 따라갔다",4,0,4,정치
449601,2016-12-14,12,jtbc,"[팩트체크] 개헌, 정말로 '두 달 반' 안에 가능할까?",1119,30,1149,정치
449602,2016-12-14,12,jtbc,"[Talk쏘는 정치] ""걸핏하면 시위해"" 촛불 폄하한 교수",6,15,21,정치


In [125]:
%%time
from konlpy.tag import Twitter
twitter = Twitter()
def words(sentence):
    word_list = twitter.nouns(sentence)
    return " ".join(word_list)

df['title_noun'] = df['title'].apply(words)
#df.reset_index().to_json('data/twitter.json',orient='records')

CPU times: user 4min 28s, sys: 460 ms, total: 4min 28s
Wall time: 4min 26s


In [64]:
df_by_month = df.groupby(['month'])
df_by_media_month = df.groupby(['month', 'media'])

In [65]:
df_by_month.agg(['mean', 'count'])

org_count        naver_count        total_count       
             mean  count        mean  count        mean  count
month                                                         
01     165.487125  40466   23.049745  40466  188.536870  40466
02     168.984964  37111   25.549163  37111  194.534127  37111
03     126.101200  40761   22.099114  40761  148.200314  40761
04     151.722805  37378   25.495265  37378  177.218069  37378
05     133.163774  39695   24.974405  39695  158.138179  39695
06     121.357529  41303   22.615137  41303  143.972665  41303
07     143.167743  43680   25.951809  43680  169.119551  43680
08     178.262277  41094   29.442595  41094  207.704872  41094
09     202.969311  33171   31.705858  33171  234.675168  33171
10     278.142503  36315   35.430015  36315  313.572518  36315
11     356.033372  40543   44.977012  40543  401.010384  40543
12     328.621973  18086   36.790335  18086  365.412308  18086

In [66]:
df_01 = df[df['month']=='01']
len(df_01)

40466

In [127]:
%%time
from collections import Counter

## df 방식과 counter 방식의 현저한 속도차
#result = df['title_noun'].head(20000).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
#result.sort_values(axis=0, ascending=False)
r1 = df['title_noun'].tolist()
result = Counter(" ".join(r1).split(" "))

CPU times: user 1.12 s, sys: 12 ms, total: 1.14 s
Wall time: 1.13 s


In [105]:
result.most_common(30)

[('년', 18928),
 ('대통령', 16965),
 ('명', 15821),
 ('종합', 14621),
 ('일', 14241),
 ('한국', 11139),
 ('더', 9769),
 ('뉴스', 9642),
 ('월', 8721),
 ('것', 8436),
 ('만', 7751),
 ('전', 7531),
 ('안', 7231),
 ('등', 6892),
 ('정부', 6867),
 ('서울', 6822),
 ('첫', 6801),
 ('억', 6774),
 ('중국', 6424),
 ('논란', 6311),
 ('중', 6294),
 ('최순실', 6267),
 ('영상', 6213),
 ('위', 5846),
 ('단독', 5832),
 ('제', 5819),
 ('트럼프', 5683),
 ('세계', 5587),
 ('박', 5551),
 ('차', 5529)]

In [107]:
by_month = df_by_month['title_noun'].apply(list)

In [115]:
result_by_month = pd.Series()

for month, words in by_month.iteritems():
    result_by_month.set_value(month, Counter(" ".join(words).split(" "))) 

In [116]:
for month, words in result_by_month.iteritems():
    print(month, words.most_common(30))

01 [('년', 1940), ('종합', 1487), ('명', 1389), ('더', 1232), ('일', 1212), ('한국', 1174), ('대통령', 909), ('첫', 832), ('중국', 816), ('뉴스', 773), ('것', 752), ('민주', 749), ('월', 689), ('위안부', 687), ('만', 674), ('경제', 662), ('전', 659), ('서울', 651), ('정부', 639), ('북한', 611), ('등', 607), ('억', 579), ('세계', 577), ('중', 573), ('논란', 567), ('한파', 549), ('미국', 532), ('안', 524), ('안철수', 523), ('위', 501)]
02 [('년', 1596), ('종합', 1255), ('명', 1191), ('일', 1084), ('더', 1038), ('한국', 907), ('개성', 882), ('공단', 882), ('대통령', 737), ('뉴스', 686), ('중국', 672), ('사드', 668), ('북한', 666), ('월', 664), ('것', 659), ('정부', 643), ('만', 628), ('안', 626), ('미사일', 618), ('민주', 617), ('첫', 574), ('등', 535), ('중', 534), ('전', 533), ('서울', 531), ('테러', 514), ('세계', 511), ('억', 508), ('제재', 497), ('위', 494)]
03 [('년', 1610), ('더', 1490), ('명', 1391), ('공천', 1338), ('일', 1160), ('종합', 1151), ('민주', 1049), ('한국', 971), ('총선', 885), ('뉴스', 827), ('김종인', 822), ('알파', 807), ('것', 800), ('대통령', 754), ('월', 719), ('후보', 690), ('위', 685

In [128]:
%%time
from konlpy.tag import Mecab
mecab = Mecab()

def mecab_words(sentence):
    word_list = mecab.nouns(sentence)
    return " ".join(word_list)

df['title_me_noun'] = df['title'].apply(mecab_words)


CPU times: user 32.9 s, sys: 16 ms, total: 32.9 s
Wall time: 32.9 s


In [129]:
by_me_month = df_by_month['title_me_noun'].apply(list)
result_by_me_month = pd.Series()

for month, words in by_me_month.iteritems():
    result_by_me_month.set_value(month, Counter(" ".join(words).split(" "))) 

In [130]:
for month, words in result_by_me_month.iteritems():
    print(month, words.most_common(30))

01 [('년', 2149), ('만', 1836), ('명', 1749), ('원', 1528), ('일', 1524), ('종합', 1493), ('대', 1150), ('한국', 1147), ('北', 921), ('대통령', 913), ('억', 891), ('중국', 845), ('민주', 804), ('위', 798), ('경제', 786), ('중', 768), ('것', 766), ('뉴스', 761), ('中', 760), ('美', 731), ('월', 696), ('서울', 692), ('위안부', 681), ('정부', 660), ('등', 633), ('세계', 616), ('북한', 610), ('시장', 602), ('국민', 578), ('논란', 570)]
02 [('년', 1818), ('만', 1740), ('명', 1549), ('北', 1412), ('원', 1290), ('일', 1282), ('종합', 1252), ('대', 983), ('한국', 916), ('개성', 882), ('공단', 879), ('美', 818), ('억', 741), ('대통령', 737), ('중', 710), ('뉴스', 689), ('중국', 688), ('것', 677), ('中', 673), ('정부', 670), ('월', 669), ('사드', 666), ('북한', 662), ('국민', 659), ('민주', 650), ('미사일', 649), ('위', 618), ('테러', 583), ('서울', 560), ('등', 555)]
03 [('년', 1779), ('명', 1726), ('만', 1616), ('일', 1434), ('공천', 1341), ('원', 1300), ('대', 1229), ('종합', 1151), ('민주', 1070), ('한국', 994), ('北', 975), ('총선', 881), ('국민', 874), ('것', 824), ('김종인', 820), ('뉴스', 819), ('억', 817

In [124]:
df.reset_index().to_json('data/mecab.json',orient='records')